In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format ='retina'

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    gpu_num = 1
    try:
        tf.config.experimental.set_visible_devices(gpus[gpu_num], 'GPU')
        #tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[gpu_num],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
    except RuntimeError as e:
        print(e)

In [4]:
from data_import_preprocessing import import_data_preprocessing
from model_define import CNNLSTM_model

In [5]:
train_data_name = 'data/nb_data_changed_HT_1-1.csv'
test_data_name = 'data/nb_data_changed_HT_1-2.csv'
preprocessing = import_data_preprocessing(train_data_file_name = train_data_name,
                                          test_data_file_name= test_data_name,
                                         )
print('train_data : ', preprocessing.train_data_file_sample_column)
print('train_data : ', preprocessing.test_data_file_sample_column)

train_data :  ['target', 'target_20bp', 'indel_percent', 'indel_rate']
train_data :  ['target', 'target_20bp', 'indel_percent', 'indel_rate']


In [6]:
data = preprocessing(sgRNA_column='target',
                    indel_column='indel_percent',
                    split_data=0.1
                    )
print(data.keys())
print(data['train'].keys())

dict_keys(['train', 'val', 'total', 'test'])
dict_keys(['seq', 'indel_rate', 'indel_class', 'read_cnt', 'info'])


In [7]:
X_train = data['train']['seq']
class_train = data['train']['indel_class']
rate_train = data['train']['indel_rate']

X_val = data['val']['seq']
class_val = data['val']['indel_class']
rate_val = data['val']['indel_rate']

X_test = data['test']['seq']
class_test = data['test']['indel_class']
rate_test = data['test']['indel_rate']

input_shape = X_train.shape[1:]

In [8]:
CNNLSTM = CNNLSTM_model(input_shape=input_shape)

In [11]:
CNNLSTM_model = CNNLSTM.MTL_model()
CNNLSTM_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 34, 4)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 34, 128)      640         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 34, 128)      1152        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 34, 128)      1664        input_1[0][0]                    
____________________________________________________________________________________________

In [13]:
CNNLSTM_model.compile(optimizer='adam',
              loss={
                  #'num_mis': 'categorical_crossentropy',
                  'class_1': 'categorical_crossentropy',
                  'class_2': 'categorical_crossentropy',
                  'class_final': 'categorical_crossentropy',
                  'rate': 'mean_squared_error'},
              loss_weights={
                  #'num_mis': 0.5,
                  'class_1': 1,
                  'class_2': 1,
                  'class_final': 0.5,
                  'rate': 1},
              metrics={
                  #'num_mis': 'accuracy',
                  'class_1': "accuracy",
                  'class_2': "accuracy",
                  'class_final': "accuracy"}
             )

In [14]:
from sklearn.utils import class_weight
class_train_num = class_train.argmax(axis=-1)
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(class_train_num),
                                                  class_train_num
                                                 )
class_weights_dict = dict(enumerate(class_weights))#{ i : class_weights[i] for i in range(11)}
print("class_weight")
print(class_weights_dict)

class_wieghts_dict_tuned = class_weights_dict
for i in range(0,11):
    multiple_constant = 2
    cutoff_class = 5
    
    if i <=cutoff_class:
        class_wieghts_dict_tuned[i] = class_weights_dict[i]#1.0
    else:
        class_wieghts_dict_tuned[i] = class_weights_dict[i]#multiple_constant*np.tanh((class_weights_dict[i]-2)/2) + multiple_constant
    
print("sample_weight")
print(class_wieghts_dict_tuned)

sample_weight = np.array([class_wieghts_dict_tuned[i] for i in class_train_num])

class_weight
{0: 0.4392529446215917, 1: 0.9573110197135158, 2: 1.1437770058459713, 3: 1.2115229291932155, 4: 1.149131767109295, 5: 1.0498483552375768, 6: 1.040943789035392, 7: 0.9701760689902983, 8: 1.0793955384984408, 9: 1.4303877940241576, 10: 1.8538862949739083}
sample_weight
{0: 0.4392529446215917, 1: 0.9573110197135158, 2: 1.1437770058459713, 3: 1.2115229291932155, 4: 1.149131767109295, 5: 1.0498483552375768, 6: 1.040943789035392, 7: 0.9701760689902983, 8: 1.0793955384984408, 9: 1.4303877940241576, 10: 1.8538862949739083}


In [17]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  min_delta=0.0005,
                                                  patience=50, verbose=0, mode='auto')

CNNLSTM_model.fit(x=X_train,
              y={
                  #'num_mis': num_mis_train,
                  'class_1': class_train,
                  'class_2': class_train,
                  'class_final': class_train,
                  'rate': rate_train},
              validation_data=(X_val, {#'num_mis': num_mis_val,
                                       'class_1': class_val,
                                       'class_2': class_val,
                                       'class_final': class_val,
                                       'rate': rate_val}),
              class_weight={
                  'class_1' : class_weights_dict,
                  'class_2' : class_weights_dict,
                  'class_final' : class_weights_dict},
              sample_weight={'rate' : sample_weight},
              shuffle=True,
              epochs=200,
              batch_size=128,
              verbose=1,
              callbacks=[early_stopping]
             )

Train on 13500 samples, validate on 1500 samples
Epoch 1/200
13500/13500 [==============================] - 18s 1ms/sample - loss: 6.1785 - class_1_loss: 2.4143 - class_2_loss: 2.4144 - class_final_loss: 2.4112 - rate_loss: 0.1439 - class_1_accuracy: 0.0883 - class_2_accuracy: 0.0893 - class_final_accuracy: 0.0901 - val_loss: 6.1356 - val_class_1_loss: 2.4101 - val_class_2_loss: 2.4103 - val_class_final_loss: 2.4101 - val_rate_loss: 0.1071 - val_class_1_accuracy: 0.0940 - val_class_2_accuracy: 0.0700 - val_class_final_accuracy: 0.0967
Epoch 2/200
13500/13500 [==============================] - 6s 481us/sample - loss: 6.1038 - class_1_loss: 2.3997 - class_2_loss: 2.4001 - class_final_loss: 2.3984 - rate_loss: 0.1045 - class_1_accuracy: 0.0872 - class_2_accuracy: 0.0845 - class_final_accuracy: 0.0807 - val_loss: 6.1360 - val_class_1_loss: 2.4106 - val_class_2_loss: 2.4107 - val_class_final_loss: 2.4100 - val_rate_loss: 0.1067 - val_class_1_accuracy: 0.0700 - val_class_2_accuracy: 0.0700 -

13500/13500 [==============================] - 7s 492us/sample - loss: 6.0953 - class_1_loss: 2.3990 - class_2_loss: 2.3991 - class_final_loss: 2.3991 - rate_loss: 0.1003 - class_1_accuracy: 0.1430 - class_2_accuracy: 0.0960 - class_final_accuracy: 0.1030 - val_loss: 6.1397 - val_class_1_loss: 2.4103 - val_class_2_loss: 2.4104 - val_class_final_loss: 2.4103 - val_rate_loss: 0.1109 - val_class_1_accuracy: 0.2073 - val_class_2_accuracy: 0.0807 - val_class_final_accuracy: 0.0700
Epoch 18/200
13500/13500 [==============================] - 7s 490us/sample - loss: 6.0950 - class_1_loss: 2.3978 - class_2_loss: 2.3978 - class_final_loss: 2.3978 - rate_loss: 0.0998 - class_1_accuracy: 0.0864 - class_2_accuracy: 0.0776 - class_final_accuracy: 0.0838 - val_loss: 6.1406 - val_class_1_loss: 2.4102 - val_class_2_loss: 2.4103 - val_class_final_loss: 2.4102 - val_rate_loss: 0.1121 - val_class_1_accuracy: 0.0847 - val_class_2_accuracy: 0.0700 - val_class_final_accuracy: 0.0700
Epoch 19/200
13500/13500 

Epoch 34/200
13500/13500 [==============================] - 7s 486us/sample - loss: 6.0943 - class_1_loss: 2.3982 - class_2_loss: 2.3982 - class_final_loss: 2.3982 - rate_loss: 0.0991 - class_1_accuracy: 0.0886 - class_2_accuracy: 0.0919 - class_final_accuracy: 0.0782 - val_loss: 6.1404 - val_class_1_loss: 2.4103 - val_class_2_loss: 2.4103 - val_class_final_loss: 2.4103 - val_rate_loss: 0.1117 - val_class_1_accuracy: 0.0847 - val_class_2_accuracy: 0.0700 - val_class_final_accuracy: 0.0847
Epoch 35/200
13500/13500 [==============================] - 7s 492us/sample - loss: 6.0947 - class_1_loss: 2.3988 - class_2_loss: 2.3988 - class_final_loss: 2.3990 - rate_loss: 0.0993 - class_1_accuracy: 0.1130 - class_2_accuracy: 0.1052 - class_final_accuracy: 0.0893 - val_loss: 6.1407 - val_class_1_loss: 2.4104 - val_class_2_loss: 2.4103 - val_class_final_loss: 2.4102 - val_rate_loss: 0.1119 - val_class_1_accuracy: 0.0760 - val_class_2_accuracy: 0.0760 - val_class_final_accuracy: 0.0700
Epoch 36/200

Epoch 51/200
13500/13500 [==============================] - 7s 499us/sample - loss: 6.0941 - class_1_loss: 2.3982 - class_2_loss: 2.3982 - class_final_loss: 2.3982 - rate_loss: 0.0990 - class_1_accuracy: 0.0740 - class_2_accuracy: 0.0730 - class_final_accuracy: 0.0744 - val_loss: 6.1398 - val_class_1_loss: 2.4102 - val_class_2_loss: 2.4103 - val_class_final_loss: 2.4102 - val_rate_loss: 0.1112 - val_class_1_accuracy: 0.1000 - val_class_2_accuracy: 0.1000 - val_class_final_accuracy: 0.1000
